Generiamo del testo dando in input un dataset txt 


In [ ]:
!pip install tensorflow
!pip install numpy


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing

import numpy as np
import os
import time


In [ ]:
#path_to_file = tf.keras.utils.get_file("digital.txt", "https://raw.githubusercontent.com/carloocchiena/rnn_text_generation/main/dataset/digitransf.txt" )

path_to_file = tf.keras.utils.get_file("metal.txt", "https://raw.githubusercontent.com/carloocchiena/rnn_text_generation/main/dataset/metal.txt" )

#when refreshing the dataset, clean also the keras temp folder C:\Users\Carlo\.keras\datasets

221184/219236 [==============================] - 0s 0us/step


In [ ]:
text = open(path_to_file, "rb").read().decode(encoding="utf-8")
print(f"Lenght of text: {len(text)} characthers")
print (text[:250])

Lenght of text: 219101 characthers
1. Hit The Lights

[James Hetfield]
[Lars Ulrich]

No life till leather
We are gonna kick some ass tonight
We got the metal madness
When our fans start screaming
It's right well alright
When we start to rock
We never want to stop again




In [ ]:
vocab = sorted(set(text))
print (f"{len(vocab)} unique characters")

89 unique characters


In [ ]:
example_texts =["abcdefghijklmnopqrstuvw", "xyz"]
chars = tf.strings.unicode_split(example_texts, input_encoding="UTF-8")

In [ ]:
ids_from_chars = preprocessing.StringLookup(vocabulary=list(vocab), mask_token=None)

In [ ]:
ids = ids_from_chars(chars)

In [ ]:
chars_from_ids = tf.keras.layers.experimental.preprocessing.StringLookup(vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

In [ ]:
chars  = chars_from_ids(ids)

In [ ]:
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefghijklmnopqrstuvw', b'xyz'], dtype=object)

In [ ]:
def text_from_ids(ids):
    return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

In [ ]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, "UTF-8"))
all_ids

<tf.Tensor: shape=(219101,), dtype=int64, numpy=array([17, 14,  3, ..., 72,  2,  1])>

In [ ]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [ ]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

1
.
 
H
i
t
 
T
h
e


In [ ]:
seq_length = 100
example_per_epoch = len(text)//(seq_length+1)

In [ ]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
    print(chars_from_ids(seq))

tf.Tensor(
[b'1' b'.' b' ' b'H' b'i' b't' b' ' b'T' b'h' b'e' b' ' b'L' b'i' b'g'
 b'h' b't' b's' b'\r' b'\n' b'\r' b'\n' b'[' b'J' b'a' b'm' b'e' b's' b' '
 b'H' b'e' b't' b'f' b'i' b'e' b'l' b'd' b']' b'\r' b'\n' b'[' b'L' b'a'
 b'r' b's' b' ' b'U' b'l' b'r' b'i' b'c' b'h' b']' b'\r' b'\n' b'\r' b'\n'
 b'N' b'o' b' ' b'l' b'i' b'f' b'e' b' ' b't' b'i' b'l' b'l' b' ' b'l'
 b'e' b'a' b't' b'h' b'e' b'r' b'\r' b'\n' b'W' b'e' b' ' b'a' b'r' b'e'
 b' ' b'g' b'o' b'n' b'n' b'a' b' ' b'k' b'i' b'c' b'k' b' ' b's' b'o'
 b'm' b'e' b' '], shape=(101,), dtype=string)


In [ ]:
for seq in sequences.take(5):
    print(text_from_ids(seq).numpy())

b'1. Hit The Lights\r\n\r\n[James Hetfield]\r\n[Lars Ulrich]\r\n\r\nNo life till leather\r\nWe are gonna kick some '
b"ass tonight\r\nWe got the metal madness\r\nWhen our fans start screaming\r\nIt's right well alright\r\nWhen w"
b'e start to rock\r\nWe never want to stop again\r\n\r\nHit the lights\r\nHit the lights\r\nHit the lights\r\n\r\nYou'
b' know our fans are insane\r\nWe are gonna blow this place away\r\nwith volume higher\r\nThan anything today'
b' the only way\r\nWhen we start to rock\r\nWe never want to stop again\r\n\r\nHit the lights\r\nHit the lights\r\n'


In [ ]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [ ]:
dataset = sequences.map(split_input_target)

In [ ]:
for input_example, target_example in dataset.take(1):
    print("Input: ", text_from_ids(input_example).numpy())
    print("Target: ",text_from_ids(target_example).numpy())

Input:  b'1. Hit The Lights\r\n\r\n[James Hetfield]\r\n[Lars Ulrich]\r\n\r\nNo life till leather\r\nWe are gonna kick some'
Target:  b'. Hit The Lights\r\n\r\n[James Hetfield]\r\n[Lars Ulrich]\r\n\r\nNo life till leather\r\nWe are gonna kick some '


In [ ]:
BATCH_SIZE = 64

BUFFER_SIZE = 10000

dataset = (dataset
           .shuffle(BUFFER_SIZE)
           .batch(BATCH_SIZE, drop_remainder=True)
           .prefetch(tf.data.experimental.AUTOTUNE))

In [ ]:
dataset

<PrefetchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [ ]:
vocab_size = len(vocab)
embedding_dim = 256
rnn_units = 1024

In [ ]:
class MyModel(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, rnn_units):
        super().__init__(self)
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(rnn_units, return_sequences=True, return_state=True)
        self.dense = tf.keras.layers.Dense(vocab_size)
        
    def call(self, inputs, states=None, return_state=False, training=False):
        x = inputs
        x = self.embedding(x, training=training)
        if states is None:
            states = self.gru.get_initial_state(x)
        x, states = self.gru(x, initial_state=states, training=training)
        x = self.dense(x, training=training)
        
        if return_state:
            return x, states
        else:
            return x

In [ ]:
model = MyModel(vocab_size=len(ids_from_chars.get_vocabulary()), embedding_dim=embedding_dim, rnn_units=rnn_units)

In [ ]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print (example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 90) # (batch_size, sequence_length, vocab_size)


In [ ]:
model.summary()

Model: "my_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  23040     
_________________________________________________________________
gru (GRU)                    multiple                  3938304   
_________________________________________________________________
dense (Dense)                multiple                  92250     
Total params: 4,053,594
Trainable params: 4,053,594
Non-trainable params: 0
_________________________________________________________________


In [ ]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [ ]:
sampled_indices

array([56, 23, 81, 25, 66, 89, 58, 28, 85,  1, 41, 83, 60, 17, 71, 73, 54,
       35, 82, 45, 70, 52,  2, 15, 40, 77, 22, 71, 55, 11, 33, 78, 54, 49,
       60, 62, 38, 71, 51, 68, 10,  1, 45,  7, 50, 14, 73, 11, 80, 67, 64,
       30, 20, 38, 14, 26, 79, 51,  0, 81, 87, 41,  1, 89,  7, 83, 73, 85,
       16, 18, 23, 69, 75, 36, 42, 38, 49,  1,  1, 20,  1, 19, 58, 63, 55,
        1, 39, 19, 18,  1, 70, 26, 55, 29, 53,  0, 30,  3, 82,  2])

In [ ]:
print ("input: \n", text_from_ids(input_example_batch[0].numpy()))
print ()
print ("Next Char Predictions: \n", text_from_ids(sampled_indices).numpy())

input: 
 tf.Tensor(b' will be heard)\r\n\r\nThrough the worst we prevail\r\nSo our voices will be heard\r\nThrough the worst we p', shape=(), dtype=string)

Next Char Predictions: 
 b']7w9h\xe2\x9f\x93`>\xc3\xa9\nLyb1moZFxPlW\r/Ks6m[*DtZTbdImVj)\nP&U.o*vifA4I.:uV[UNK]w\xe2\x80\x99L\n\xe2\x9f\x93&yo\xc3\xa9027kqGMIT\n\n4\n3`e[\nJ32\nl:[?Y[UNK]A x\r'


In [ ]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [ ]:
example_batch_loss = loss(target_example_batch, example_batch_predictions)
mean_loss = example_batch_loss.numpy().mean()
print ("Prediction shape: ", example_batch_predictions.shape, "# (batch_size, sequence_lenght, vocab_size)")
print ("Mean loss:        ", mean_loss)

Prediction shape:  (64, 100, 90) # (batch_size, sequence_lenght, vocab_size)
Mean loss:         4.4996147


In [ ]:
tf.exp(mean_loss).numpy()

89.98245

A newly initialized model shouldn't be too sure of itself, the output logits should all have similar magnitudes. To confirm this you can check that the exponential of the mean loss is approximately equal to the vocabulary size. A much higher loss means the model is sure of its wrong answers, and is badly initialized:

In [ ]:
vocab_size   

89

In [ ]:
model.compile(optimizer="adam", loss=loss, metrics=["accuracy"])

In [ ]:
#directory where checkpoints are saved
checkpoint_dir = "./training_checkpoints"
#name of checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_prefix, save_weights_only=True)

In [ ]:
EPOCHS = 600


history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/600
33/33 [==============================] - 4s 56ms/step - loss: 3.8523 - accuracy: 0.1574
Epoch 2/600
33/33 [==============================] - 2s 55ms/step - loss: 2.7380 - accuracy: 0.2829
Epoch 3/600
33/33 [==============================] - 2s 54ms/step - loss: 2.3601 - accuracy: 0.3424
Epoch 4/600
33/33 [==============================] - 2s 55ms/step - loss: 2.2321 - accuracy: 0.3616
Epoch 5/600
33/33 [==============================] - 2s 54ms/step - loss: 2.1399 - accuracy: 0.3805
Epoch 6/600
33/33 [==============================] - 2s 55ms/step - loss: 2.0544 - accuracy: 0.3998
Epoch 7/600
33/33 [==============================] - 2s 55ms/step - loss: 1.9728 - accuracy: 0.4207
Epoch 8/600
33/33 [==============================] - 2s 55ms/step - loss: 1.8942 - accuracy: 0.4404
Epoch 9/600
33/33 [==============================] - 2s 56ms/step - loss: 1.8191 - accuracy: 0.4611
Epoch 10/600
33/33 [==============================] - 2s 56ms/step - loss: 1.7484 - accuracy: 0.4795

In [ ]:
class OneStep(tf.keras.Model):
    def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
        super().__init__()
        self.temperature = temperature
        self.model = model
        self.chars_from_ids = chars_from_ids
        self.ids_from_chars = ids_from_chars
        
        #create a mask to prevent [UNK] from bein generated
        skip_ids = self.ids_from_chars(["[UNK]"])[:,None]
        sparse_mask = tf.SparseTensor(
        #Put a -inf at each bad index
        values=[-float("inf")]*len(skip_ids),
        indices=skip_ids,
        #match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
        self.prediction_mask = tf.sparse.to_dense(sparse_mask)
        
    @tf.function
    def generate_one_step(self, inputs, states=None):
        #convert strings to token IDs
        input_chars = tf.strings.unicode_split(inputs, "UTF-8")
        input_ids = self.ids_from_chars(input_chars).to_tensor()
        
        #run the model
        #predicted_logits.shape is [batch, char, next_char_logits]
        predicted_logits, states = self.model(inputs=input_ids, states=states, return_state=True)
        
        #use only the last prediction
        predicted_logits = predicted_logits [:, -1, :]
        predicted_logits = predicted_logits/self.temperature
        #apply the prediction mask, prevent "[UNK]" from bein generated
        predicted_logits = predicted_logits + self.prediction_mask
        
        #sample the output logits to generate token IDs
        predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
        predicted_ids = tf.squeeze(predicted_ids, axis=1)
        
        #convert from token ids to characters
        predicted_chars = self.chars_from_ids(predicted_ids)
        
        #return the characthers and model state
        return predicted_chars, states
    

In [ ]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [ ]:
start = time.time()
states = None
next_char = tf.constant(["3. Vanity"])
result = [next_char]

for n in range(1000):
    next_char, states = one_step_model.generate_one_step(next_char, states=states)
    result.append(next_char)
    
result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode("utf-8"), "\n\n" + "_"*80)
print("\nRun time:", end - start)

3. Vanity
Ain't be our hidowa no wrush to foose show Agay not with the Mort

When I wantion lade

[Jamal Made. ArYost of coming
My songremoking to me?
Respect the weaking
an out time

So someoncen to me
I know and shattered to my jedge.
What is your name
Your cangight until that's reading
you. You have burn lives my lorg on he'll lies are conflew if you.
Hatce its there, yeah.

Cause to thom for nears
Love me that I'm time

Felling the ternen before
You have to some meture

I’'s do begrage the thostend's warke makes
esce seoning when I say, you'll never beon a will be heard
Thun times you. Do It Ride taken their start
Fall

Follow me that's readinate
i' lose
Can't here I disfing?

Buit does, I've home
those off there
Your burness and hight better that been in privare,
to swose own my a frarituce it means
You have to the wrath arrock
See, dour that I murt now a man
Make it right
Trustion the one much more I take?

Mone, punks enough to keep unhthis world  

_____________________________

#### Export the generator
This single-step model can  be saved and restored, allowing you to use it anywhere a tf.saved_model is accepted.
Run these cells when needed only.

In [ ]:
'''
tf.saved_model.save(one_step_model, 'one_step')
one_step_reloaded = tf.saved_model.load('one_step')
'''

"\ntf.saved_model.save(one_step_model, 'one_step')\none_step_reloaded = tf.saved_model.load('one_step')\n"

In [ ]:
'''
states = None
next_char = tf.constant(["Digital"])
result = [next_char]

for n in range(100):
    next_char, states = one_step_reloaded.generate_one_step(next_char, states=states)
    result.append(next_char)
    
print (tf.strings.join(result)[0].numpy().decode("utf-8"))
'''

'\nstates = None\nnext_char = tf.constant(["Digital"])\nresult = [next_char]\n\nfor n in range(100):\n    next_char, states = one_step_reloaded.generate_one_step(next_char, states=states)\n    result.append(next_char)\n    \nprint (tf.strings.join(result)[0].numpy().decode("utf-8"))\n'